## **Land usage classification in Agriculture**


## Abstract


We will explore how land use types can be classified based on aerial and satellite photographs of the earth's surface. We will learn to build our own photo masks based on ready-made masks and display them on the screen.


## Introduction


Determining how land is used is a huge problem today. After all, its improper and illegal use can lead to both economic and natural disasters. One of the ways to assess the use is the analysis of aerial and satellite images of the earth's surface. A big problem is to build a mathematical model that can determine the type of land use based on colors. If you have ready-made photos and masks of land use, you can use the methods of artificial intelligence and big data to build a model-classifier.

The statistical data was obtained from https://www.kaggle.com/humansintheloop/semantic-segmentation-of-aerial-imagery under [CC0: Public Domain](https://creativecommons.org/publicdomain/zero/1.0/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsdatascienceinagriculturelanduseclassification462-2022-01-01) license.


# Download and visualization of images


## Install the required libraries


We need to install additional libraries and upgrade existing ones in the lab.


In [2]:
!pip install matplotlib>=3.4

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install opencv-python

In [ ]:
!pip install colormap

In [ ]:
!pip install scikit-learn-intelex

## Import the required libraries


We will use libraries os and glob to work with files and folders.
We will also use Matplotlib and Seaborn for visualizing our dataset to gain a better understanding of the images we are going to be handling.
NumPy will be used for arrays of images. Scikit-Learn - for classical classification models. Pandas - for DataSet creation.


In [18]:
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import json
from PIL import Image
from colormap import rgb2hex, hex2rgb

#Classifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay

## Loading of data


Create a function that downloads all and displays the last images of land and masks from a specified directory.
All training pictures and their masks have to be placed in differend directories.
Separate a csv file that contains the description of land use classes.

The function has to work in the following way:
1. Download a csv file **[json.load()](https://docs.python.org/3/library/json.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsdatascienceinagriculturelanduseclassification462-2022-01-01)** and display classes description.
2. Download all images from a specific directory:
 **[Image.open()](https://pillow.readthedocs.io/en/stable/reference/Image.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsdatascienceinagriculturelanduseclassification462-2022-01-01)**.
3. Display the last image + mask.
4. Form and return a DataSet that has to be an array of tuples [image, mask].

Remark: the downloaded directories contain 72 images and their masks located in different subfolders according to image resolutions. Due to the fact that the dataset for training will represent each point in the form of a separate record - the size of the dataset will be too large to conduct the training on a local computer. Therefore, we will select a separate folder that contains 9 photos and their masks.


In [ ]:
!pip install skillsnetwork

import skillsnetwork

await skillsnetwork.prepare("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/data-science-in-agriculture-land-use-classification/Semantic_segmentation_dataset.zip", overwrite=True)

In [36]:
def get_data(folder, file):
    # download json
    f = open(folder + "/" + file,)
    data = json.load(f)
    f.close()
    cl = {}
    # Create a dictionary with classes
    for i, c in enumerate(data['classes']):
        cl[i] = dict(c)

    for k, v in cl.items():
        print('Class', k)
        for k2, v2 in v.items():
            print("   ", k2, v2)
    data = []

    # download images
    sd = [item for item in os.listdir(folder) if os.path.isdir(folder + '/' + item)] # a list of subdirectories
    print("Subdirectories: ", sd)
    for f in [sd[-1]]:  # Choose the last subdirectory to download
        print("Downloading: ", f)
        images = glob(folder + "/" + f + "/images" + "/*.jpg") # create a list of image files
        for im in images:
            mask_f = im.replace("images", "masks").replace("jpg", "png") # create a list of mask files
            image = Image.open(im)
            mask = Image.open(mask_f)
            if len(np.array(mask).shape) > 2:
                data.append([image, mask])
        fig = plt.figure(figsize = (10,10)) #display the last image + mask
        plt.subplot(1,2,1)
        plt.imshow(image)
        plt.subplot(1,2,2)
        plt.imshow(mask)
        plt.show()

    return data

In [ ]:
d = "Semantic segmentation dataset"
f = "classes.json"
data = get_data(d, f)

Downloaded all the pictures and their masks from a separate directory and formed a set of data lists consisting of tuples of picture-mask.


# DataSet creation


Let's see how many images we downloaded:


In [ ]:
len(data)

Using 40% images and masks for training and 60% for tests.

Make a function that will create a DataSet.

Each image is a set of points. Each point is represented by a tuple of RGB (red, green, blue). Every color is a number [0-1) for float or [0, 255) for int.
Therefore, every image is a 3D array (height, width, color). Or a 2D array for gray scale.

To establish the dependence of color -> land use class, there's need to convert each image into a dataset of the form (r, g, b) -> class.

To do this, transform the image into a color matrix **[np.asarray()](https://numpy.org/doc/stable/reference/generated/numpy.asarray.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsdatascienceinagriculturelanduseclassification462-2022-01-01)**, and then transform it into a one-dimensional form **[np.flatten()](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flatten.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsdatascienceinagriculturelanduseclassification462-2022-01-01)**.
To construct the output field, we need to additionally convert the color tuple (r, g, b) from the mask file into hex format: **[rgb2hex()](https://pythonhosted.org/colormap/references.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsdatascienceinagriculturelanduseclassification462-2022-01-01)**.


In [39]:
def create_DataSet(data):
    DS = pd.DataFrame()
    for image, mask in data:
        # transform image to matrix
        im = np.asarray(image)
        mk = np.asarray(mask)
        # transform a one-dimension array of r, g, b colors
        red = im[:,:,0].flatten()
        green = im[:,:,1].flatten()
        blue = im[:,:,2].flatten()
        im_f = np.array([red, green, blue])
        red = mk[:,:,0].flatten()
        green = mk[:,:,1].flatten()
        blue = mk[:,:,2].flatten()
        # calculate hex classes
        h = np.array([rgb2hex(*m) for m in zip(red, green, blue)])
        mk_f = np.array([red, green, blue, h])
        d = np.concatenate((im_f, mk_f), axis=0)
        # create a DataSet
        DS_new = pd.DataFrame(np.transpose(d), columns = ['Im_Red', 'Im_Green', 'Im_Blue', 'Mk_Red', 'Mk_Green', 'Mk_Blue', 'HEX'])
        if len(DS) == 0:
            DS = DS_new
        else:
            DS = DS.append(DS_new)
    return DS

In [ ]:
print("Create a training DataSet")
train = create_DataSet(data[:4])
print(train)
print("Create a test DataSet")
test = create_DataSet(data[4:])
print(test)

Study the column types of training and test DataSets.


In [ ]:
train.info()

In [ ]:
test.info()

All the columns have object type.

The last column 'HEX' contains colors in hex format. Therefore, it is necessary to change the type of this data to categorical.


In [ ]:
train.loc[:, 'HEX'] = train['HEX'].astype('category')
train['HEX']

In [ ]:
test.loc[:, 'HEX'] =  test['HEX'].astype('category')
test['HEX']

All other columns contain colors in int format, therefore we should change their types:


In [ ]:
cl = ['Im_Red', 'Im_Green', 'Im_Blue', 'Mk_Red', 'Mk_Green', 'Mk_Blue']
train[cl] = train[cl].astype('int64')
test[cl] = test[cl].astype('int64')
print (train.info())
print (test.info())

Visualize the data and see what exactly we are working with. Use seaborn to plot the number of pixel classes and you can see what the output looks like.


In [ ]:
c = pd.DataFrame(train['HEX'])
sns.set_style('darkgrid')
sns.countplot(x="HEX", data=c)

In [ ]:
c = pd.DataFrame(test['HEX'])
sns.set_style('darkgrid')
sns.countplot(x="HEX", data=c)

The training and test DataSets contain similar distribution of images classes.


# Classification model creation


Use **[sklearn.linear_model.LogisticRegression()](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsdatascienceinagriculturelanduseclassification462-2022-01-01)** classifier for mask analysis. It is a very fast and simple classifier. In the next lab we will compare different classifiers.

Using the first 3 columns (RGB of image pixel) as imput parameters, and the last column (HEX color of mask picture) as an output.

The **[fit()](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsdatascienceinagriculturelanduseclassification462-2022-01-01&highlight=logisticregression#sklearn.linear_model.LogisticRegression.fit)** and **[score()](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logisticregression#sklearn.linear_model.LogisticRegression.score)** functions are used for training and evaluating the accuracy.


We will use function **[ConfusionMatrixDisplay.from_estimator()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html)** for analysis.


In [ ]:
clf = LogisticRegression(max_iter=500, n_jobs=-1)
c = train.columns
clf.fit(train[c[0:3]], train[c[-1:]].values.ravel())

In [ ]:
scores_train = clf.score(train[c[0:3]], train[c[-1:]].values.ravel())
scores_test = clf.score(test[c[0:3]], test[c[-1:]].values.ravel())
print('Accuracy train DataSet: {: .1%}'.format(scores_train), 'Accuracy test DataSet: {: .1%}'.format(scores_test))
ConfusionMatrixDisplay.from_estimator(clf, test[c[0:3]], test[c[-1:]].values.ravel())
plt.show()

As you can see, the accuracy is not bad. The difference between the training and test sets is little. It means that the model is not bad, and for increasing the accuracy we should increase our DataSet. You can test it yourself by adding all the directories with images.


# Create your own mask of land use


To build a mask based on the classifier model, you would need to choose a few images from the downloaded data list and build your DataSet.


In [ ]:
test_image = 8 # choose the number of images from the data list
mask_test = data[test_image:test_image+1] # Test Image + Mask
mask_test_DataSet = create_DataSet(mask_test) #Build a DataSet
print(mask_test_DataSet)

Then, calculate the hex colour of classes using the model:


In [ ]:
c = mask_test_DataSet.columns
mask_test_predict = clf.predict(mask_test_DataSet[c[0:3]])
print(mask_test_predict)

Further, there's need to transform the 1D array of colors into a 2D matrix of Hex color using **[numpy.reshape()](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsdatascienceinagriculturelanduseclassification462-2022-01-01)**.


In [ ]:
size = mask_test[0][1].size #get original image size
print(size)
predict_img = np.array(mask_test_predict).reshape((size[1], size[0])) #reshaping array of HEX colour
print(predict_img)

Than we create a 3D image matrix with an RGB colour map using **[colormap.hex2rgb()](https://pythonhosted.org/colormap/references.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsdatascienceinagriculturelanduseclassification462-2022-01-01)**.


In [ ]:
rgb_size = np.array(mask_test[0][0]).shape
print("Image size: ", rgb_size)
predict_img_rgb = np.zeros(rgb_size)
for i, r in enumerate(predict_img):
    for j, c in enumerate(r):
        predict_img_rgb[i, j, 0], predict_img_rgb[i, j, 1], predict_img_rgb[i, j, 2] = hex2rgb(c)

Let's compare the mask with the original one.


In [ ]:
predict_img_rgb = predict_img_rgb.astype('int')
print("Model mask")
plt.imshow(predict_img_rgb)
plt.show()
print("Real mask")
plt.imshow(mask_test[0][1])
plt.show()

You can see that these masks are very similar and you just have to increase the DataSets to improve the accuracy.


## Conclusions


Created an expert system based on classifiers, which allows one to obtain the classification of land, based on aero and satellite images. These principles can be used for any type of images and any types of land use.

Uploaded and converted images. Mastered extracting image colors and building / training / testing sets of classifiers. Calculated the accuracy and studied how to build a mask based on a classifier.


## Authors


[Elomunait John Omoding](https://www.linkedin.com/in/johnelomunait)

[Yaroslav Vyklyuk, prof., PhD., DrSc](https://author.skills.network/instructors/yaroslav_vyklyuk_2)


 Copyright &copy; 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsdatascienceinagriculturelanduseclassification462-2022-01-01).
